In [1]:
import os
import pandas as pd
import random
from pysolar import solar
import datetime
import reverse_geocoder as rg 
from wwo_hist import retrieve_hist_data

In [2]:
now = datetime.datetime.now(datetime.timezone.utc)
date_format = "%b %d %Y %H"
total_minutes_in_year = 60*24*365
total_solar_panel_area = 1
minute_diff = 60


#Turkey's latitude and longitude ranges
min_latitude = 35.9025
max_latitude = 42.0268
min_longitude = 25.9090
max_longitude = 44.5742

In [3]:
def get_altitude_by_location(lat,lon,date):
    return solar.get_altitude(lat, lon, date)

def get_azimuth_by_location(lat,lon,date):
    return solar.get_azimuth(lat, lon, date)

def get_radiation_by_location(lat,lon,date):
    return solar.radiation.get_radiation_direct(date, get_altitude_by_location(lat,lon,date))

In [4]:
def get_solar_info(lat,lon,date):
    return get_altitude_by_location(lat,lon,date),\
            get_azimuth_by_location(lat,lon,date),\
            get_radiation_by_location(lat,lon,date)




In [5]:
def get_location(coordinates):
    return rg.search(coordinates) 

In [6]:
def get_random_location(min_lat,max_lat,min_lon,max_lon):
     return random.uniform(min_lat,max_lat), random.uniform(min_lon,max_lon)

location = get_random_location(min_latitude,max_latitude,min_longitude,max_longitude)
print("Coordinates", location)
result = get_location(location)
print("City name is" , result[0]["admin1"], "\nCounty name is" ,result[0]["name"])
latitude, longitude = location[0],location[1]

Coordinates (39.03371571317861, 27.915631380825776)
Loading formatted geocoded file...
City name is Manisa 
County name is Akhisar


In [7]:
dates = []
solar_info = []
for min_diff in range(total_minutes_in_year,0,minute_diff*-1):
    past_date = now - datetime.timedelta(minutes=min_diff)
    dates.append(past_date)
    solar_info.append(get_solar_info(latitude,longitude,past_date))

/Users/egehangunduz/anaconda3/lib/python3.7/site-packages/pysolar/solartime.py:112: UserWarning: I don't know about leap seconds after 2018
  (leap_seconds_base_year + len(leap_seconds_adjustments) - 1)


In [8]:
df = pd.concat([pd.DataFrame(dates,columns=["date"]),\
                pd.DataFrame(solar_info,columns=["altitude","azimuth","solar_radiation"])],\
               axis=1)

In [9]:
df.set_index("date",inplace=True)

In [13]:
frequency = 1
start_date = now - datetime.timedelta(minutes=total_minutes_in_year)
end_date = now
api_key = os.environ['WWO_API_KEY'] 
location_list = [str(latitude) +  "," + str(longitude)]

In [16]:
hist_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                store_df = True)



Retrieving weather data for 39.03371571317861,27.915631380825776


Currently retrieving data for 39.03371571317861,27.915631380825776: from 2018-12-15 to 2018-12-31
Time elapsed (hh:mm:ss.ms) 0:00:01.916020
Currently retrieving data for 39.03371571317861,27.915631380825776: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:00:03.594797
Currently retrieving data for 39.03371571317861,27.915631380825776: from 2019-02-01 to 2019-02-28
Time elapsed (hh:mm:ss.ms) 0:00:04.825274
Currently retrieving data for 39.03371571317861,27.915631380825776: from 2019-03-01 to 2019-03-31
Time elapsed (hh:mm:ss.ms) 0:00:06.058864
Currently retrieving data for 39.03371571317861,27.915631380825776: from 2019-04-01 to 2019-04-30
Time elapsed (hh:mm:ss.ms) 0:00:07.409804
Currently retrieving data for 39.03371571317861,27.915631380825776: from 2019-05-01 to 2019-05-31
Time elapsed (hh:mm:ss.ms) 0:00:09.211417
Currently retrieving data for 39.03371571317861,27.915631380825776: from 2019-06-01 to 2019

In [17]:
hist_data

[             date_time maxtempC mintempC totalSnow_cm sunHour uvIndex uvIndex  \
 0  2018-12-15 00:00:00       14        7          0.0     8.7       1       1   
 1  2018-12-15 01:00:00       14        7          0.0     8.7       1       1   
 2  2018-12-15 02:00:00       14        7          0.0     8.7       1       1   
 3  2018-12-15 03:00:00       14        7          0.0     8.7       1       1   
 4  2018-12-15 04:00:00       14        7          0.0     8.7       1       1   
 5  2018-12-15 05:00:00       14        7          0.0     8.7       1       1   
 6  2018-12-15 06:00:00       14        7          0.0     8.7       1       1   
 7  2018-12-15 07:00:00       14        7          0.0     8.7       1       1   
 8  2018-12-15 08:00:00       14        7          0.0     8.7       1       1   
 9  2018-12-15 09:00:00       14        7          0.0     8.7       1       1   
 10 2018-12-15 10:00:00       14        7          0.0     8.7       1       1   
 11 2018-12-15 1

In [18]:
df

,altitude,azimuth,solar_radiation
date,,,
2018-12-15 19:15:29.747729+00:00,-51.180737,281.286499,0.000000
2018-12-15 20:15:29.747729+00:00,-62.206426,296.518039,0.000000
2018-12-15 21:15:29.747729+00:00,-71.236105,323.505001,0.000000
2018-12-15 22:15:29.747729+00:00,-74.060047,9.986443,0.000000
2018-12-15 23:15:29.747729+00:00,-68.083522,49.379213,0.000000
2018-12-16 00:15:29.747729+00:00,-57.961789,70.385621,0.000000
2018-12-16 01:15:29.747729+00:00,-46.627050,83.301965,0.000000
2018-12-16 02:15:29.747729+00:00,-34.999557,93.096928,0.000000
2018-12-16 03:15:29.747729+00:00,-23.462641,101.731832,0.000000
